In [3]:
from datamodel import (
    Listing,
    Observation,
    Order,
    OrderDepth,
    ProsperityEncoder,
    UserId,
    Symbol,
    Trade,
    TradingState,
)
from typing import List
import copy
import numpy as np
import math
from itertools import permutations
import Round_5

###############
import sys
import os

sys.path.append(os.path.abspath("../"))

import webbrowser
from datetime import datetime
from pathlib import Path
from typing import Any, Optional
from importlib import reload
from collections import defaultdict
from functools import partial, reduce
from http.server import HTTPServer, SimpleHTTPRequestHandler

from prosperity3bt.data import has_day_data, read_day_data
from prosperity3bt.file_reader import (
    FileReader,
    FileSystemReader,
    PackageResourcesReader,
)
from prosperity3bt.models import BacktestResult
from prosperity3bt.runner import run_backtest
################


import json
from typing import Any

In [4]:
class tradable_product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    DJEMBES = "DJEMBES"
    VOLCANIC_ROCK = "VOLCANIC_ROCK"
    VOLCANIC_ROCK_VOUCHER_9500 = "VOLCANIC_ROCK_VOUCHER_9500"
    VOLCANIC_ROCK_VOUCHER_9750 = "VOLCANIC_ROCK_VOUCHER_9750"
    VOLCANIC_ROCK_VOUCHER_10000 = "VOLCANIC_ROCK_VOUCHER_10000"
    VOLCANIC_ROCK_VOUCHER_10250 = "VOLCANIC_ROCK_VOUCHER_10250"
    VOLCANIC_ROCK_VOUCHER_10500 = "VOLCANIC_ROCK_VOUCHER_10500"
    MAGNIFICENT_MACARONS = "MAGNIFICENT_MACARONS"


class position_limit:
    KELP = 50
    RAINFOREST_RESIN = 50
    SQUID_INK = 50
    CROISSANTS = 250
    JAMS = 350
    PICNIC_BASKET1 = 60
    PICNIC_BASKET2 = 100
    DJEMBES = 60
    VOLCANIC_ROCK = 400
    VOLCANIC_ROCK_VOUCHER_9500 = 200
    VOLCANIC_ROCK_VOUCHER_9750 = 200
    VOLCANIC_ROCK_VOUCHER_10000 = 200
    VOLCANIC_ROCK_VOUCHER_10250 = 200
    VOLCANIC_ROCK_VOUCHER_10500 = 200
    MAGNIFICENT_MACARONS = 75
    MAGNIFICENT_MACARONS_CONVERSION = 10


class traders:
    Caesar = "Caesar"
    Camilla = "Camilla"
    Charlie = "Charlie"
    Gary = "Gary"
    Gina = "Gina"
    Olga = "Olga"
    Olivia = "Olivia"
    Pablo = "Pablo"
    Paris = "Paris"
    Penelope = "Penelope"
    Peter = "Peter"

In [5]:
def parse_data(data_root: Optional[Path]) -> FileReader:
    if data_root is not None:
        return FileSystemReader(data_root)
    else:
        return PackageResourcesReader()


def parse_days(file_reader: FileReader, days: list[str]) -> list[tuple[int, int]]:
    parsed_days = []

    for arg in days:
        if "-" in arg:
            round_num, day_num = map(int, arg.split("-", 1))

            if not has_day_data(file_reader, round_num, day_num):
                print(f"Warning: no data found for round {round_num} day {day_num}")
                continue

            parsed_days.append((round_num, day_num))
        else:
            round_num = int(arg)

            parsed_days_in_round = []
            for day_num in range(-5, 6):
                if has_day_data(file_reader, round_num, day_num):
                    parsed_days_in_round.append((round_num, day_num))

            if len(parsed_days_in_round) == 0:
                print(f"Warning: no data found for round {round_num}")
                continue

            parsed_days.extend(parsed_days_in_round)

    if len(parsed_days) == 0:
        print("Error: did not find data for any requested round/day")
        sys.exit(1)

    return parsed_days


def parse_out(out: Optional[Path], no_out: bool) -> Optional[Path]:
    if out is not None:
        return out

    if no_out:
        return None

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return Path.cwd() / "backtests" / f"{timestamp}.log"


def print_day_summary(result: BacktestResult) -> None:
    last_timestamp = result.activity_logs[-1].timestamp

    product_lines = []
    total_profit = 0

    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        product = row.columns[2]
        profit = row.columns[-1]

        product_lines.append(f"{product}: {profit:,.0f}")
        total_profit += profit

    print(*reversed(product_lines), sep="\n")
    print(f"Total profit: {total_profit:,.0f}")


def merge_results(
    a: BacktestResult,
    b: BacktestResult,
    merge_profit_loss: bool,
    merge_timestamps: bool,
) -> BacktestResult:
    sandbox_logs = a.sandbox_logs[:]
    activity_logs = a.activity_logs[:]
    trades = a.trades[:]

    if merge_timestamps:
        a_last_timestamp = a.activity_logs[-1].timestamp
        timestamp_offset = a_last_timestamp + 100
    else:
        timestamp_offset = 0

    sandbox_logs.extend([row.with_offset(timestamp_offset) for row in b.sandbox_logs])
    trades.extend([row.with_offset(timestamp_offset) for row in b.trades])

    if merge_profit_loss:
        profit_loss_offsets = defaultdict(float)
        for row in reversed(a.activity_logs):
            if row.timestamp != a_last_timestamp:
                break

            profit_loss_offsets[row.columns[2]] = row.columns[-1]

        activity_logs.extend(
            [
                row.with_offset(timestamp_offset, profit_loss_offsets[row.columns[2]])
                for row in b.activity_logs
            ]
        )
    else:
        activity_logs.extend(
            [row.with_offset(timestamp_offset, 0) for row in b.activity_logs]
        )

    return BacktestResult(a.round_num, a.day_num, sandbox_logs, activity_logs, trades)


def write_output(output_file: Path, merged_results: BacktestResult) -> None:
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with output_file.open("w+", encoding="utf-8") as file:
        file.write("Sandbox logs:\n")
        for row in merged_results.sandbox_logs:
            file.write(str(row))

        file.write("\n\n\nActivities log:\n")
        file.write(
            "day;timestamp;product;bid_price_1;bid_volume_1;bid_price_2;bid_volume_2;bid_price_3;bid_volume_3;ask_price_1;ask_volume_1;ask_price_2;ask_volume_2;ask_price_3;ask_volume_3;mid_price;profit_and_loss\n"
        )
        file.write("\n".join(map(str, merged_results.activity_logs)))

        file.write("\n\n\n\n\nTrade History:\n")
        file.write("[\n")
        file.write(",\n".join(map(str, merged_results.trades)))
        file.write("]")


def print_overall_summary(results: list[BacktestResult]) -> None:
    print("Profit summary:")

    total_profit = 0
    for result in results:
        last_timestamp = result.activity_logs[-1].timestamp

        profit = 0
        for row in reversed(result.activity_logs):
            if row.timestamp != last_timestamp:
                break

            profit += row.columns[-1]

        print(f"Round {result.round_num} day {result.day_num}: {profit:,.0f}")
        total_profit += profit

    print(f"Total profit: {total_profit:,.0f}")

    return total_profit


def format_path(path: Path) -> str:
    cwd = Path.cwd()
    if path.is_relative_to(cwd):
        return str(path.relative_to(cwd))
    else:
        return str(path)


class HTTPRequestHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        self.server.shutdown_flag = True
        return super().do_GET()

    def end_headers(self) -> None:
        self.send_header("Access-Control-Allow-Origin", "*")
        return super().end_headers()

    def log_message(self, format: str, *args: Any) -> None:
        return


class CustomHTTPServer(HTTPServer):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.shutdown_flag = False


def open_visualizer(output_file: Path) -> None:
    http_handler = partial(HTTPRequestHandler, directory=str(output_file.parent))
    http_server = CustomHTTPServer(("localhost", 0), http_handler)

    webbrowser.open(
        f"https://jmerle.github.io/imc-prosperity-3-visualizer/?open=http://localhost:{http_server.server_port}/{output_file.name}"
    )

    while not http_server.shutdown_flag:
        http_server.handle_request()

In [6]:
all_data = []
file_reader = parse_data(
    Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
    )
)
days = parse_days(file_reader, [str(day) for day in range(3, 6)])
for round_num, day_num in days:
    all_data.append(read_day_data(file_reader, round_num, day_num, no_names=True))

In [7]:
def main(all_data, params: dict, output_file=None) -> None:
    results = []
    file_reader = parse_data(
        Path(
            "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/prosperity3bt/resources"
        )
    )
    days = parse_days(file_reader, [str(day) for day in range(3, 6)])
    for i in range(len(days)):
        round_num, day_num = days[i]
        reload(Round_5)
        data = all_data[i]
        result = run_backtest(
            trader=Round_5.Trader(params),
            data=data,
            file_reader=file_reader,
            round_num=round_num,
            day_num=day_num,
            print_output=False,
            trade_matching_mode="all",
            no_names=True,  # args.no_names,
            show_progress_bar=False,
        )

        # print_day_summary(result)
        if len(days) > 1:
            print()

        results.append(result)

    if output_file is not None:
        merged_results = reduce(
            lambda a, b: merge_results(
                a, b, merge_profit_loss=True, merge_timestamps=True
            ),
            results,
        )
        write_output(output_file, merged_results)
        print(f"\nSuccessfully saved backtest results to {format_path(output_file)}")
        open_visualizer(output_file)

    final_profit = print_overall_summary(results)
    return final_profit

In [32]:
params = {
    tradable_product.RAINFOREST_RESIN: {
        "ask_slip": 1,
        "bid_slip": -1,
        "decay": 0.1,
        "threshold": 0.0,
        "trade_with": {
            traders.Pablo: [4, 5],
            traders.Caesar: [1],
            traders.Charlie: [7],
            traders.Penelope: [8],
        },
        "trade_against": {
            traders.Paris: [1],
            traders.Charlie: [5, 8],
            traders.Penelope: [1, 6, 7],
        },
    },
    tradable_product.KELP: {
        "alpha": np.array([[-1.93181848e-04], [0.0]]),
        "beta": np.array([[1.0, -0.9341909]]),
        "gamma": np.array(
            [
                [-2.37870411e-02, -3.37620662e-01],
                [0.0, -4.70692887e-01],
            ]
        ),
        "decay": 0.6,
        "threshold": 0.8,
        "trade_with": {
            traders.Gina: [i for i in range(11)],
            traders.Paris: [1],
            traders.Camilla: [1],
            traders.Pablo: [i for i in range(6)],
            traders.Caesar: [i for i in range(2, 7)],
            traders.Penelope: [2, 3, 4, 5, 14],
        },
        "trade_against": {
            traders.Paris: [2],
            traders.Caesar: [1],
            traders.Charlie: [i for i in range(10)],
            traders.Penelope: [6, 10, 11],
        },
    },
    tradable_product.SQUID_INK: {
        "alpha": np.array([[-1.93181848e-04], [0.0]]),
        "beta": np.array([[1.0, -0.9341909]]),
        "gamma": np.array(
            [
                [-2.37870411e-02, -3.37620662e-01],
                [0.0, -4.70692887e-01],
            ]
        ),
        "z_threshold": 0,
        "trade_with": {
            traders.Gina: [i for i in range(11)],
            traders.Paris: [1],
            traders.Camilla: [1, 2, 6],
            traders.Olivia: [
                1,
                14,
                15,
            ],
            traders.Pablo: [i for i in range(6)],
            traders.Caesar: [2, 3, 4, 5, 6],
            traders.Charlie: [
                13,
                11,
            ],
            traders.Gary: [1],
            traders.Penelope: [1, 2, 3, 4, 12, 14],
        },
        "trade_against": {
            traders.Paris: [2],
            traders.Caesar: [1],
            traders.Charlie: [
                1,
                7,
                2,
                5,
                9,
                4,
                6,
                3,
                10,
                14,
            ],
            traders.Gary: [12, 13],
            traders.Penelope: [6, 10, 11, 15],
        },
    },
    tradable_product.PICNIC_BASKET1: {
        tradable_product.CROISSANTS: 6,
        tradable_product.JAMS: 3,
        tradable_product.DJEMBES: 1,
        "alpha": np.array([[-5.61674869e-05], [0.0]]),
        "beta": np.array([[1.0, -0.07318334]]),
        "gamma": np.array([[-0.11347602, 0.00577044], [0.39925558, -0.03051414]]),
        "z_threshold": 1.8,
        "z_threshold2": 1.6,
        "exit_threshold": 2.4,
        "trade_with": {
            traders.Pablo: [
                1,
                7,
                2,
                5,
                6,
                3,
                8,
            ],
            traders.Penelope: [8],
        },
        "trade_against": {
            traders.Camilla: [1, 2, 3, 5, 6, 7, 8],
            traders.Caesar: [1, 2, 5, 7, 8, 10],
        },
    },
    tradable_product.PICNIC_BASKET2: {
        tradable_product.CROISSANTS: 4,
        tradable_product.JAMS: 2,
        tradable_product.DJEMBES: 0,
        "z_threshold": 2,
        "exit_threshold": 2,
        "trade_with": {
            traders.Pablo: [i for i in range(8)],
            traders.Caesar: [1],
            traders.Charlie: [10],
        },
        "trade_against": {
            traders.Camilla: [
                1,
                7,
                8,
                2,
                6,
                3,
            ],
            traders.Caesar: [5, 8],
            traders.Charlie: [i for i in range(8)],
        },
    },
    tradable_product.CROISSANTS: {
        "alpha": np.array([[-5.61674869e-05], [0.0]]),
        "beta": np.array([[1.0, -0.07318334]]),
        "gamma": np.array([[-0.11347602, 0.00577044], [0.39925558, -0.03051414]]),
        "z_threshold": 6,
        "trade_with": {
            traders.Paris: [7, 8],
            traders.Camilla: [i for i in range(11)],
            traders.Olivia: [3],
            traders.Caesar: [3],
        },
        "trade_against": {
            traders.Paris: [5],
            traders.Caesar: [i for i in range(4, 11)],
        },
    },
    tradable_product.JAMS: {
        "alpha": np.array([[-2.56244811e-04], [0.0]]),
        "beta": np.array([[1.0, -0.11116249]]),
        "gamma": np.array([[0.0, 0.00482966], [0.26875689, -0.03614998]]),
        "z_threshold": 3.8,
        "trade_with": {traders.Camilla: [i for i in range(12)]},
        "trade_against": {traders.Caesar: [i for i in range(12)]},
    },
    tradable_product.DJEMBES: {
        "alpha": np.array([[-9.90008420e-05], [1.71206399e-03]]),
        "beta": np.array([[1.0, -0.22870816]]),
        "gamma": np.array([[0.03207502, 0.0], [0.11032108, -0.02182072]]),
        "z_threshold": 1,
        "trade_with": {
            traders.Paris: [5],
            traders.Camilla: [i for i in range(3, 12)],
            traders.Caesar: [2],
        },
        "trade_against": {
            traders.Paris: [
                2,
            ],
            traders.Caesar: [i for i in range(3, 11)],
        },
    },
    tradable_product.VOLCANIC_ROCK: {
        "hedge_pos": 0,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Pablo: [
                1,
            ]
        },
        "trade_against": {
            traders.Caesar: [
                1,
            ]
            + [i for i in range(5, 11)]
        },
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_9500: {
        "strike": 9500,
        "dte": 3e6,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Pablo: [i for i in range(21)],
            traders.Caesar: [
                3,
                5,
            ]
            + [i for i in range(10, 21)],
            traders.Penelope: [1, 2, 11, 15, 16],
        },
        "trade_against": {
            traders.Camilla: [i for i in range(21)],
            traders.Caesar: [1, 2, 6, 7],
            traders.Penelope: [
                12,
            ],
        },
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_9750: {
        "strike": 9750,
        "dte": 3e6,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Pablo: [i for i in range(21)],
            traders.Caesar: [
                6,
            ]
            + [i for i in range(10, 21)],
            traders.Penelope: [1, 2, 6, 7, 9, 8, 11, 15],
        },
        "trade_against": {
            traders.Camilla: [i for i in range(21)],
            traders.Caesar: [
                1,
                7,
                2,
                4,
                6,
            ],
            traders.Penelope: [1, 4, 10, 12, 19],
        },
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10000: {
        "strike": 10_000,
        "dte": 3e6,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Camilla: [
                3,
            ],
            traders.Pablo: [i for i in range(20)],
            traders.Caesar: [5, 6] + [i for i in range(19, 21)],
            traders.Penelope: [1, 5, 6, 7, 15, 11],
        },
        "trade_against": {
            traders.Camilla: [2] + [i for i in range(5, 21)],
            traders.Caesar: [
                1,
                7,
                2,
            ],
            traders.Penelope: [12],
        },
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10250: {
        "strike": 10_250,
        "dte": 3e6,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Pablo: [i for i in range(21)],
            traders.Caesar: [i for i in range(10, 21)],
            traders.Penelope: [
                3,
                6,
            ],
        },
        "trade_against": {
            traders.Camilla: [
                2,
            ]
            + [i for i in range(5, 21)],
            traders.Caesar: [
                1,
                7,
                2,
                5,
                6,
                3,
            ],
        },
    },
    tradable_product.VOLCANIC_ROCK_VOUCHER_10500: {
        "strike": 10_500,
        "dte": 3e6,
        "z_threshold": 0.55,
        "trade_with": {
            traders.Pablo: [i for i in range(21)],
            traders.Caesar: [i for i in range(10, 21)],
        },
        "trade_against": {
            traders.Camilla: [i for i in range(5, 21)],
            traders.Caesar: [
                1,
                2,
                5,
                7,
            ],
        },
    },
    tradable_product.MAGNIFICENT_MACARONS: {
        "alpha": np.array([[-0.11809086], [0.02087891], [-0.00018677]]),
        "beta": np.array([[1.0, -1.00619334, -0.04519032]]),
        "gamma": np.array(
            [
                [-4.43562692e-01, 4.41261197e-01, -5.54291956e00],
                [-1.28555496e-02, 9.24099084e-03, -3.89429168e00],
                [1.43503499e-04, -1.58118913e-04, 6.27232006e-01],
            ],
        ),
        "z_threshold": 1.8,
        "trade_with": {
            traders.Paris: [i for i in range(1, 5)],
            traders.Charlie: [i for i in range(4, 8)],
        },
        "trade_against": {
            traders.Paris: [i for i in range(6, 8)],
            traders.Camilla: [i for i in range(2, 8)],
            traders.Caesar: [
                2,
                5,
                4,
                6,
            ],
            traders.Charlie: [1, 2],
        },
    },
}


result = main(
    all_data=all_data,
    params=params,
    output_file=Path(
        "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/Round 5/test.log"
    ),
)





Successfully saved backtest results to test.log
Profit summary:
Round 5 day 2: -221,351
Round 5 day 3: -156,612
Round 5 day 4: -334,063
Total profit: -712,026


In [9]:
# param_range = np.arange(-1, 1, 0.25)
# all_combi = list(permutations(param_range, 2))

best = 0
perm = {"PNL": [], "i": [], "j": []}

print(f"Permuations Count: {len(list(permutations(np.arange(1, 3, 0.2), 2)))}")

for i in [1.28, 1.64, 1.96, 2.33, 2.58]:
    for j in [1.28, 1.64, 1.96, 2.33, 2.58]:
        # j = 3
        window = 0.3
        threshold = 0
        params = {
            tradable_product.RAINFOREST_RESIN: {
                "ask_slip": 1,
                "bid_slip": -1,
                "decay": 0.1,
                "threshold": 0.0,
                "trade_with": {
                    traders.Pablo: [4, 5],
                    traders.Caesar: [1],
                    traders.Charlie: [7],
                    traders.Penelope: [8],
                },
                "trade_against": {
                    traders.Paris: [1],
                    traders.Charlie: [5, 8],
                    traders.Penelope: [1, 6, 7],
                },
            },
            tradable_product.KELP: {
                "alpha": np.array([[-1.93181848e-04], [0.0]]),
                "beta": np.array([[1.0, -0.9341909]]),
                "gamma": np.array(
                    [
                        [-2.37870411e-02, -3.37620662e-01],
                        [0.0, -4.70692887e-01],
                    ]
                ),
                "decay": 0.6,
                "threshold": 0.8,
                "trade_with": {
                    traders.Gina: [i for i in range(11)],
                    traders.Paris: [1],
                    traders.Camilla: [1],
                    traders.Pablo: [i for i in range(6)],
                    traders.Caesar: [i for i in range(2, 7)],
                    traders.Penelope: [2, 3, 4, 5, 14],
                },
                "trade_against": {
                    traders.Paris: [2],
                    traders.Caesar: [1],
                    traders.Charlie: [i for i in range(10)],
                    traders.Penelope: [6, 10, 11],
                },
            },
            tradable_product.SQUID_INK: {
                "alpha": np.array([[-1.93181848e-04], [0.0]]),
                "beta": np.array([[1.0, -0.9341909]]),
                "gamma": np.array(
                    [
                        [-2.37870411e-02, -3.37620662e-01],
                        [0.0, -4.70692887e-01],
                    ]
                ),
                "z_threshold": 0,
                "trade_with": {
                    traders.Gina: [i for i in range(11)],
                    traders.Paris: [1],
                    traders.Camilla: [1, 2, 6],
                    traders.Olivia: [
                        1,
                        14,
                        15,
                    ],
                    traders.Pablo: [i for i in range(6)],
                    traders.Caesar: [2, 3, 4, 5, 6],
                    traders.Charlie: [
                        13,
                        11,
                    ],
                    traders.Gary: [1],
                    traders.Penelope: [1, 2, 3, 4, 12, 14],
                },
                "trade_against": {
                    traders.Paris: [2],
                    traders.Caesar: [1],
                    traders.Charlie: [
                        1,
                        7,
                        2,
                        5,
                        9,
                        4,
                        6,
                        3,
                        10,
                        14,
                    ],
                    traders.Gary: [12, 13],
                    traders.Penelope: [6, 10, 11, 15],
                },
            },
            tradable_product.PICNIC_BASKET1: {
                tradable_product.CROISSANTS: 6,
                tradable_product.JAMS: 3,
                tradable_product.DJEMBES: 1,
                "alpha": np.array([[-5.61674869e-05], [0.0]]),
                "beta": np.array([[1.0, -0.07318334]]),
                "gamma": np.array(
                    [[-0.11347602, 0.00577044], [0.39925558, -0.03051414]]
                ),
                "z_threshold": i,
                "z_threshold2": 1.64,
                "exit_threshold": j,
                "trade_with": {
                    traders.Pablo: [
                        1,
                        7,
                        2,
                        5,
                        6,
                        3,
                        8,
                    ],
                    traders.Penelope: [8],
                },
                "trade_against": {
                    traders.Camilla: [1, 2, 3, 5, 6, 7, 8],
                    traders.Caesar: [1, 2, 5, 7, 8, 10],
                },
            },
            tradable_product.PICNIC_BASKET2: {
                tradable_product.CROISSANTS: 4,
                tradable_product.JAMS: 2,
                tradable_product.DJEMBES: 0,
                "z_threshold": i,
                "exit_threshold": j,
                "trade_with": {
                    traders.Pablo: [i for i in range(8)],
                    traders.Caesar: [1],
                    traders.Charlie: [10],
                },
                "trade_against": {
                    traders.Camilla: [
                        1,
                        7,
                        8,
                        2,
                        6,
                        3,
                    ],
                    traders.Caesar: [5, 8],
                    traders.Charlie: [i for i in range(8)],
                },
            },
            tradable_product.CROISSANTS: {
                "alpha": np.array([[-5.61674869e-05], [0.0]]),
                "beta": np.array([[1.0, -0.07318334]]),
                "gamma": np.array(
                    [[-0.11347602, 0.00577044], [0.39925558, -0.03051414]]
                ),
                "z_threshold": 6,
                "trade_with": {
                    traders.Paris: [7, 8],
                    traders.Camilla: [i for i in range(11)],
                    traders.Olivia: [3],
                    traders.Caesar: [3],
                },
                "trade_against": {
                    traders.Paris: [5],
                    traders.Caesar: [i for i in range(4, 11)],
                },
            },
            tradable_product.JAMS: {
                "alpha": np.array([[-2.56244811e-04], [0.0]]),
                "beta": np.array([[1.0, -0.11116249]]),
                "gamma": np.array([[0.0, 0.00482966], [0.26875689, -0.03614998]]),
                "z_threshold": 3.8,
                "trade_with": {traders.Camilla: [i for i in range(12)]},
                "trade_against": {traders.Caesar: [i for i in range(12)]},
            },
            tradable_product.DJEMBES: {
                "alpha": np.array([[-9.90008420e-05], [1.71206399e-03]]),
                "beta": np.array([[1.0, -0.22870816]]),
                "gamma": np.array([[0.03207502, 0.0], [0.11032108, -0.02182072]]),
                "z_threshold": 1,
                "trade_with": {
                    traders.Paris: [5],
                    traders.Camilla: [i for i in range(3, 12)],
                    traders.Caesar: [2],
                },
                "trade_against": {
                    traders.Paris: [
                        2,
                    ],
                    traders.Caesar: [i for i in range(3, 11)],
                },
            },
            tradable_product.VOLCANIC_ROCK: {
                "hedge_pos": 0,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Pablo: [
                        1,
                    ]
                },
                "trade_against": {
                    traders.Caesar: [
                        1,
                    ]
                    + [i for i in range(5, 11)]
                },
            },
            tradable_product.VOLCANIC_ROCK_VOUCHER_9500: {
                "strike": 9500,
                "dte": 3e6,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Pablo: [i for i in range(21)],
                    traders.Caesar: [
                        3,
                        5,
                    ]
                    + [i for i in range(10, 21)],
                    traders.Penelope: [1, 2, 11, 15, 16],
                },
                "trade_against": {
                    traders.Camilla: [i for i in range(21)],
                    traders.Caesar: [1, 2, 6, 7],
                    traders.Penelope: [
                        12,
                    ],
                },
            },
            tradable_product.VOLCANIC_ROCK_VOUCHER_9750: {
                "strike": 9750,
                "dte": 3e6,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Pablo: [i for i in range(21)],
                    traders.Caesar: [
                        6,
                    ]
                    + [i for i in range(10, 21)],
                    traders.Penelope: [1, 2, 6, 7, 9, 8, 11, 15],
                },
                "trade_against": {
                    traders.Camilla: [i for i in range(21)],
                    traders.Caesar: [
                        1,
                        7,
                        2,
                        4,
                        6,
                    ],
                    traders.Penelope: [1, 4, 10, 12, 19],
                },
            },
            tradable_product.VOLCANIC_ROCK_VOUCHER_10000: {
                "strike": 10_000,
                "dte": 3e6,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Camilla: [
                        3,
                    ],
                    traders.Pablo: [i for i in range(20)],
                    traders.Caesar: [5, 6] + [i for i in range(19, 21)],
                    traders.Penelope: [1, 5, 6, 7, 15, 11],
                },
                "trade_against": {
                    traders.Camilla: [2] + [i for i in range(5, 21)],
                    traders.Caesar: [
                        1,
                        7,
                        2,
                    ],
                    traders.Penelope: [12],
                },
            },
            tradable_product.VOLCANIC_ROCK_VOUCHER_10250: {
                "strike": 10_250,
                "dte": 3e6,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Pablo: [i for i in range(21)],
                    traders.Caesar: [i for i in range(10, 21)],
                    traders.Penelope: [
                        3,
                        6,
                    ],
                },
                "trade_against": {
                    traders.Camilla: [
                        2,
                    ]
                    + [i for i in range(5, 21)],
                    traders.Caesar: [
                        1,
                        7,
                        2,
                        5,
                        6,
                        3,
                    ],
                },
            },
            tradable_product.VOLCANIC_ROCK_VOUCHER_10500: {
                "strike": 10_500,
                "dte": 3e6,
                "z_threshold": 0.55,
                "trade_with": {
                    traders.Pablo: [i for i in range(21)],
                    traders.Caesar: [i for i in range(10, 21)],
                },
                "trade_against": {
                    traders.Camilla: [i for i in range(5, 21)],
                    traders.Caesar: [
                        1,
                        2,
                        5,
                        7,
                    ],
                },
            },
            tradable_product.MAGNIFICENT_MACARONS: {
                "alpha": np.array([[-0.11809086], [0.02087891], [-0.00018677]]),
                "beta": np.array([[1.0, -1.00619334, -0.04519032]]),
                "gamma": np.array(
                    [
                        [-4.43562692e-01, 4.41261197e-01, -5.54291956e00],
                        [-1.28555496e-02, 9.24099084e-03, -3.89429168e00],
                        [1.43503499e-04, -1.58118913e-04, 6.27232006e-01],
                    ],
                ),
                "z_threshold": i,
                "trade_with": {
                    traders.Paris: [i for i in range(1, 5)],
                    traders.Charlie: [i for i in range(4, 8)],
                },
                "trade_against": {
                    traders.Paris: [i for i in range(6, 8)],
                    traders.Camilla: [i for i in range(2, 8)],
                    traders.Caesar: [
                        2,
                        5,
                        4,
                        6,
                    ],
                    traders.Charlie: [1, 2],
                },
            },
        }

        result = main(all_data=all_data, params=params)

        perm["PNL"].append(result)
        perm["i"].append(i)
        perm["j"].append(j)

        if result > best:
            best = result
            print(best, i, j)

print(best)


Permuations Count: 90









Profit summary:
Round 3 day 0: 8,181
Round 3 day 1: -20,612
Round 3 day 2: -12,518
Round 4 day 1: -20,612
Round 4 day 2: -12,518
Round 4 day 3: 6,048
Round 5 day 2: -12,518
Round 5 day 3: 6,048
Round 5 day 4: 22,789
Total profit: -35,711









Profit summary:
Round 3 day 0: 14,028
Round 3 day 1: -12,842
Round 3 day 2: -4,569
Round 4 day 1: -12,842
Round 4 day 2: -4,569
Round 4 day 3: -6,629
Round 5 day 2: -4,365
Round 5 day 3: -6,629
Round 5 day 4: 31,534
Total profit: -6,883









Profit summary:
Round 3 day 0: 11,915
Round 3 day 1: -12,742
Round 3 day 2: 12,288
Round 4 day 1: -12,742
Round 4 day 2: 12,288
Round 4 day 3: -1,156
Round 5 day 2: 12,288
Round 5 day 3: -1,078
Round 5 day 4: 40,057
Total profit: 61,119
61119.0 1.28 1.96









Profit summary:
Round 3 day 0: 11,125
Round 3 day 1: -10,624
Round 3 day 2: 15,886
Round 4 day 1: -10,624
Round 4 day 2: 15,886
Round 4 day 3: 269
Round 5 day 2: 15,886
Round 5 day 3: 269
Round 5 day 4: 40,955
To

In [26]:
import pandas as pd

pd.DataFrame.from_dict(perm).sort_values(by="PNL", ascending=False)

,PNL,i,j
7,52724.0,1.4,2.0
3,40325.0,1.2,2.0
15,37905.0,1.8,2.0
11,37721.0,1.6,2.0
14,28279.0,1.8,1.8
6,21238.0,1.4,1.8
10,10381.0,1.6,1.8
2,10097.0,1.2,1.8
13,4830.0,1.8,1.6
5,-7180.0,1.4,1.6
